In [ ]:
# !pip install earthengine-api
# !pip install altair
# !pip install geemap
# !pip install altair_saver

In [ ]:
# Import the required libraries

import pandas as pd
import altair as alt
import numpy as np
import folium
import ee
import geemap
from tqdm import tqdm


<p> We are using google earth engine api to pull and analize the data from the google earthengine server. For that, we first need to authenticate the earth engine api with proper google credentials. to know more about the authentication process checkout <a href="https://developers.google.com/earth-engine/apidocs/ee-authenticate"><b>Authenticator</b></a>


In [ ]:
ee.Authenticate()
ee.Initialize()

Now we will define our analysis time frame. We are considering images of 20 years earlier from the current timestamp.  

In [ ]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-20, 'years'), today)

The are of interest is the <a href="https://en.wikipedia.org/wiki/Kutupalong_refugee_camp"><b>Kutupalong Rohingya Camp</b></a>, Cox's Bazar, in Bangladesh. 

In [ ]:
aoi_shp_path =  "aoi/Kutupalong-AOI-Polygon-polygon.shp"
aoi = geemap.shp_to_ee(aoi_shp_path)
aoi

Let us check the meta data of our aoi shape file

In [ ]:
aoi.getInfo()

In [ ]:
import numpy as np
aoi_geometry = aoi.geometry()
coords = aoi_geometry.getInfo()['coordinates']
mapBoundary = coords[0][1]

coords = np.array(coords[0])
coords # the physical coordiantes of the aoi, later will be used to center the folium map

For the analysis we are using <b>Landsat 8</b> data set. There are other satellite images from different satellites. However, the spatial resolution of Landsat 8 is more suitable for our analysis. For narowing down the search we will filter it using <b> WRS ROW and PATH</b> and a time frame of 20 years from current time. Since we are doing a a timeseries anaylsis, and sometimes getting best cloud cover is uncertain, we will use a max cloud cover of 20%. 

In [ ]:
landsat =  ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
.filterMetadata('WRS_PATH','equals',136).filterMetadata('WRS_ROW', 'equals',45)\
.filterDate(date_range)\
.filter(ee.Filter.lt('CLOUD_COVER', 20))
print("Total Image Downloaded: ",landsat.size().getInfo()) #gets the total number of images

Lets check the meadata for the first image to understand what we are dealing with. Since, loading the big raster takes more time and space complexity, we are going to clip it according to our area of interest.

In [ ]:
test_image = landsat.first()
test_image = test_image.clip(aoi)

In [ ]:
test_image.getInfo()

Visualizing the test image using folium map

In [ ]:
# ##### earth-engine drawing method setup
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
  ).add_to(self)

# configuring earth engine display rendering method in folium
folium.Map.add_ee_layer = add_ee_layer

# visual parameters for the satellite imagery natural colors display
image_params = {
  'bands': ['B4',  'B3',  'B2'],
  'min': 0,
  'max': 0.3,
  'gamma': 1
}

map = folium.Map(location=[coords[:,1].mean(), coords[:,0].mean()], 
                 zoom_start=13, height=400, control_scale=True) # to ensure a perfect zooming to the area we are using
                                                                # the mean of the coordinates of our aoi
map.add_ee_layer(test_image, image_params, 'LandsatFirst')
map.add_child(folium.LayerControl())
map

We are going to use the following simple formula for calculating the NDBI. <strong><i>NDBI = (SWIR - NIR) / (SWIR + NIR)</i></strong> 

In [ ]:
# nir  = test_image.select('B4')
# swir = test_image.select('B5')


# ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')

In [ ]:
# ndbi_visParams = {min: -1, max: 1};


# map_ndbi = folium.Map(location=[coords[:,1].mean(), coords[:,0].mean()], 
#                  zoom_start=13, height=400, control_scale=True)
# map_ndbi.add_ee_layer(ndbi, ndbi_visParams, 'NDBI')

# map_ndbi.add_child(folium.LayerControl())


# map_ndbi

In [ ]:
# doi.toList(doi.size())
landsat.size().getInfo()

In [ ]:
'defining the function for calcualating the ndbi from each image. takes ee.image as input \
returns the max, min and mean NDBI from the image along with' 


def get_ndbi_info(img):
    
    img_date = img.date().getInfo()['value']
    img_sytemindex = img.get('system:index').getInfo()
  
    nir  = img.select('B4') #picking the near infrared band from the img
    swir = img.select('B5') #picking short wave infrared
    
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    
    min_ndbi_val = ndbi.reduceRegion(ee.Reducer.min(), aoi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDBI']
    max_ndbi_val = ndbi.reduceRegion(ee.Reducer.max(), aoi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDBI']
    mean_ndbi_val = ndbi.reduceRegion(ee.Reducer.mean(), aoi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDBI']
    
    img_all_info = {
      'system:index': img_sytemindex,
      'date': img_date,
      'min_ndbi' : min_ndbi_val,
      'max_ndbi' : max_ndbi_val,
      'mean_ndbi' : mean_ndbi_val      
      }

    return img_all_info


In [ ]:
type(landsat)

In [ ]:
landsat_list = landsat.toList(landsat.size()) # the landsat image collection is an imagecollection object which is 
                                                #an earthengine native datatype. this collections can not be iterated by loop. 
                                                #So we have to convert it to a list or array to iteratively process it. 
  
landsat_list_size = landsat_list.size().getInfo()

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df = pd.DataFrame(columns=['SystemIndex', 'Millis', 'MinNDBI', 'MaxNDBI', 'MeanNDBI'])

for i in tqdm(range(landsat_list_size)):
    image = ee.Image(landsat_list.get(i))
    image_info = get_ndbi_info(image)
#     print(image_info)
    df = df.append({'SystemIndex': image_info['system:index'], 
                    'Millis':  image_info['date'], 
                    'MinNDBI': image_info['min_ndbi'], 
                    'MaxNDBI': image_info['max_ndbi'], 
                    'MeanNDBI': image_info['mean_ndbi']}, 
                    ignore_index=True)

In [ ]:
df

In [ ]:
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['Millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [ ]:
df = add_date_info(df).drop(columns=['Millis'])
df

In [48]:
pd.set_option('display.max_rows', None)
df

,SystemIndex,MinNDBI,MaxNDBI,MeanNDBI,Timestamp,Year,Month,Day,DOY
0,LC08_136045_20130912,-0.501476,0.856036,0.691952,2013-09-12 04:21:02.280,2013,9,12,255
1,LC08_136045_20131014,-0.293474,0.832426,0.642996,2013-10-14 04:20:54.570,2013,10,14,287
2,LC08_136045_20131030,-0.365039,0.849545,0.705365,2013-10-30 04:20:45.650,2013,10,30,303
3,LC08_136045_20131201,-0.326832,0.789117,0.608659,2013-12-01 04:20:39.720,2013,12,1,335
4,LC08_136045_20131217,-0.41801,0.794675,0.586202,2013-12-17 04:20:32.730,2013,12,17,351
5,LC08_136045_20140102,-0.415812,0.759421,0.544641,2014-01-02 04:20:25.400,2014,1,2,2
6,LC08_136045_20140203,-0.42692,0.73186,0.499669,2014-02-03 04:20:04.710,2014,2,3,34
7,LC08_136045_20140219,-0.448789,0.754585,0.530828,2014-02-19 04:19:51.170,2014,2,19,50
8,LC08_136045_20140408,0.015119,0.468663,0.220345,2014-04-08 04:19:09.990,2014,4,8,98
9,LC08_136045_20140424,-0.107072,0.606625,0.433819,2014-04-24 04:18:53.960,2014,4,24,114


In [50]:
fig = alt.Chart(df).transform_fold(
    ['MinNDBI', 'MaxNDBI', 'MeanNDBI'],
    as_=['NDBI_Type', 'Value']
).mark_line().encode(
    x='Timestamp:T',
    y='Value:Q',
    color='NDBI_Type:N'
)

In [51]:
fig

alt.Chart(...)